In [4]:
import textdistance as td

In [7]:
td.levenshtein("Test","Test")

0

In [8]:

td.levenshtein("Test","Text")

1

In [9]:
td.jaro_winkler("this test","that test")

0.8052910052910053

In [10]:
td.jaccard('this','that')

0.3333333333333333

In [11]:
td.cosine('apple','ppale')

1.0

In [14]:
td.mra('alex','alice')

2

In [15]:
td.editex('alex','alice')

5

In [27]:
td.editex('tie','euphemism')

14

In [28]:
td.hamming.normalized_similarity('alex','alice')

0.4

In [31]:
td.hamming.normalized_similarity('tie','tie')

1.0